In [1]:
import numpy as np
import cv2
import pandas as pd

In [2]:
img = cv2.imread("Train_images/Sandstone_Versa0000.tif")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [3]:
# Create dataframe and engineer features
df = pd.DataFrame()

# Feature: original pixel values
img2 = img.reshape(-1)
df["Original Image"] = img2  

In [4]:
# Feature: first set of Gabor features by applying filters to original image
# Apply 32 generated Gabor filters w/ various parameters to image 
num = 1
kernels = []
for theta in range(2):
    theta = theta / 4. * np.pi
    for sigma in (1, 3):
        for lamda in np.arange(0, np.pi, np.pi / 4):
            for gamma in (0.05, 0.5):
                gabor_label = "Gabor" + str(num)
                ksize = 5
                kernel = cv2.getGaborKernel(
                    (ksize, ksize), 
                    sigma, theta, lamda, gamma, 0, 
                    ktype=cv2.CV_32F)
                kernels.append(kernel)
                fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img
                print(gabor_label, ": theta=", theta, ": sigma=", sigma,
                     ": lamda=", lamda, ": gamma=", gamma)
                num += 1

Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.05
Gabor2 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.5
Gabor3 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor4 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor5 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor6 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor7 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.05
Gabor8 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.5
Gabor9 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.05
Gabor10 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.5
Gabor11 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor12 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor13 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor14 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor15 : theta= 0.0 : sigma= 3 : lamda= 2

In [5]:
# Canny edge; edge detection filter
edges = cv2.Canny(img, 100, 200)
edges1 = edges.reshape(-1)
df["Canny Edge"] = edges1

In [6]:
# Some more filters
from skimage.filters import roberts, sobel, scharr, prewitt
edge_roberts = roberts(img)
edge_roberts1 = edge_roberts.reshape(-1)
df["Roberts"] = edge_roberts1

edge_sobel = sobel(img)
edge_sobel1 = edge_sobel.reshape(-1)
df["Sobel"] = edge_sobel1

edge_scharr = scharr(img)
edge_scharr1 = edge_scharr.reshape(-1)
df["Scharr"] = edge_scharr1

edge_prewitt = prewitt(img)
edge_prewitt1 = edge_prewitt.reshape(-1)
df["Prewitt"] = edge_prewitt1

In [7]:
from scipy import ndimage as nd
gaussian_img = nd.gaussian_filter(img, sigma=3)
gaussian_img1 = gaussian_img.reshape(-1)
df["Gaussian s3"] = gaussian_img1

gaussian_img2 = nd.gaussian_filter(img, sigma=7)
gaussian_img3 = gaussian_img2.reshape(-1)
df["Gaussian s7"] = gaussian_img3

median_img = nd.median_filter(img, size=3)
median_img1 = median_img.reshape(-1)
df["Median s3"] = median_img1

# variance_img = nd.generic_filter(img, np.var, size=3)
# variance_img1 = variance_img.reshape(-1)
# df["Variance s3"] = variance_img1

In [8]:
# Add a column for mask (ground truth)
labeled_img = cv2.imread("Train_masks/Sandstone_Versa0000.tif")
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img1 = labeled_img.reshape(-1)
df["Label"] = labeled_img1

In [9]:
Y = df["Label"].values
X = df.drop(labels=["Label"], axis=1)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=20)

In [11]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=100)

In [12]:
%%time
model.fit(X_train, Y_train)

Wall time: 53.1 s


C:\Users\alif-\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(max_iter=100)

In [13]:
from sklearn import metrics
prediction_test = model.predict(X_test)
print("Accuracy = ", metrics.accuracy_score(Y_test, prediction_test))

Accuracy =  0.9584544638961472


In [14]:
import pickle
filename = "sandstone_model_svm"
pickle.dump(model, open(filename, "wb"))

In [15]:
load_model = pickle.load(open(filename, "rb"))
result = load_model.predict(X)

In [16]:
segmented = result.reshape((img.shape))

In [17]:
from matplotlib import pyplot as plt
plt.imshow(segmented, cmap="turbo")
plt.imsave("versa_0000_turbo_svm.jpg", segmented, cmap="turbo")

### Train #2

In [21]:
model = LinearSVC(max_iter=1000)

In [22]:
%%time
model.fit(X_train, Y_train)

Wall time: 8min 29s


C:\Users\alif-\AppData\Roaming\Python\Python39\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC()

In [23]:
prediction_test = model.predict(X_test)
print("Accuracy = ", metrics.accuracy_score(Y_test, prediction_test))

Accuracy =  0.9644917909020938


In [24]:
filename = "sandstone_model_svm_1000"
pickle.dump(model, open(filename, "wb"))

In [25]:
load_model = pickle.load(open(filename, "rb"))
result = load_model.predict(X)

In [26]:
segmented = result.reshape((img.shape))

In [27]:
from matplotlib import pyplot as plt
plt.imshow(segmented, cmap="turbo")
plt.imsave("versa_0000_turbo_svm_1000.jpg", segmented, cmap="turbo")